## DATA

In [9]:
with open("data/all_book_data.txt") as f:
    book_data = f.read()

In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=5000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [37]:
# text_splitter.split_text(book_data)
splitted_books_data = text_splitter.split_text(book_data)

In [43]:
import nltk
num_words = [len(nltk.word_tokenize(txt)) for txt in splitted_books_data]

In [45]:
max(num_words),min(num_words)

(1158, 435)

In [55]:
def chunk_transcripts(json_data, chunk_size):
    all_chunks = {}

    for key, data in json_data.items():
        transcript_chunks = []
        current_chunk_text = ""
        current_chunk_start_time = None
        word_count = 0

        for item in data:
            text = item['text']
            start_time = item['start']

            words = text.split()

            if word_count + len(words) <= chunk_size:
                current_chunk_text += " " + text
                word_count += len(words)
                if current_chunk_start_time is None:
                    current_chunk_start_time = start_time
            else:
                transcript_chunks.append({'text': current_chunk_text.strip(), 'start_time': current_chunk_start_time})
                current_chunk_text = text
                current_chunk_start_time = start_time
                word_count = len(words)

        if current_chunk_text:
            transcript_chunks.append({'text': current_chunk_text, 'start_time': current_chunk_start_time})

        all_chunks[key] = transcript_chunks

    return all_chunks

In [56]:
import json

with open('data/misc_transcripts.json', 'r') as f:
    data = json.load(f)

In [68]:
import glob
import json
directory_path = 'data'  # Update this to your directory
json_pattern = '*.json'
json_files = glob.glob(f"data/chunked_*.json")

transcripts_text = []

for jf in json_files:
    with open(f'{jf}', 'r') as f:
        data = json.load(f)
    for youtube_id,texts in data.items():
        for text in texts:
            curr_text = text['text']
            if len(curr_text.split()) <=100: continue
            transcripts_text.append(curr_text)